In [1]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras

import os
import datetime

from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten, Conv2D, Dropout
from Chess.view import view_board_colour
from Chess.state import Board, construct_board
from Chess.constants import WHITE, BLACK

import tabulate

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, KFold, train_test_split

from ipywidgets import interact, fixed
import ipywidgets as widgets

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
np.random.seed(20)

In [3]:
%load_ext tensorboard

In [4]:
# From https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks
def new_tensorboard_run():
    logdir = os.path.join("tensorboard_logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    return tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

### Visualisation functions

In [5]:
# Functions to manipulate the output
def predict_val(model, val):
    prediction = model.predict(val)
    return np.where(prediction > 0.5, 1, 0)

def predict_raw(model, val):
    return model.predict(val)  

def find_diff(y, pred):
    Y_instance = y
    return np.where(pred == Y_instance, 1, 0)

In [6]:
# Functions to draw the labels and predictions.

sns.set_theme(style="white")
sns.color_palette("crest", as_cmap=True)
sns.set_palette("crest")

def plot_sns(val):
    # From https://seaborn.pydata.org/examples/many_pairwise_correlations.html
    sns.heatmap(val.reshape(8,8))
    plt.show()

def plot(val):
    cmap = colors.ListedColormap(['white', 'green'])
    bounds = [-0.5, 0.5, 1.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)

    fig, ax = plt.subplots()
    ax.imshow(val.reshape(8, 8), cmap=cmap, norm=norm)

    # draw gridlines
    ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
    ax.set_xticks(np.arange(0.5, 8.5, 1));
    ax.set_yticks(np.arange(0.5, 8.5, 1));
    plt.show()

def plot_diff(val):
    cmap = colors.ListedColormap(['red', 'green'])
    bounds = [-0.5, 0.5, 1.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)

    fig, ax = plt.subplots()
    ax.imshow(val.reshape(8,8) , cmap=cmap, norm=norm)

    # draw gridlines
    ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
    ax.set_xticks(np.arange(0.5, 8.5, 1));
    ax.set_yticks(np.arange(0.5, 8.5, 1));
    plt.show()

In [7]:
def generate_plots(
            n, 
            model = None,
            x = None,
            y = None,
            target=True, 
            prediction=True, 
            raw_prediction=True, 
            diff=True
        ):
    if target:         
        plot(y[n].reshape(1, 64))    
    if prediction:     
        plot(predict_val(model, x[n:n+1]))
    if raw_prediction: 
        plot_sns(predict_raw(model, x[n:n+1]))
    if diff:          
        pred = predict_val(model, x[n:n+1])
        plot_diff(find_diff(y[n:n+1], pred))

In [8]:
def grid_search(build_fn, param_grid):
    model = KerasClassifier(build_fn=build_fn, verbose=0)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)
    grid_Result = grid.fit(X, Y)
    
    print(f"best result: {grid_result.best_score_} using {grid_result.best_params_}")
    means = grid_result.cv_results_['mean_test_score']
    sdev = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for m, s, p in zip(means, sdev, params):
        print(f"{m} ({s}) with: {p}")
        

def disp_grid(grid_result):
    print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
#     for mean, stdev, param in zip(means, stds, params):
#         print("%f (%f) with: %r" % (mean, stdev, param))
    res = []
    for m, s, p in zip(means, stds, params):
        res.append({
            "mean": m,
            "stdev": s,
            **p
        })
    head = {
        "mean": "Mean",
        "stdev": "Standard deviation"
    }
    for p in params[0]:
        head[p] = p
    print(tabulate.tabulate(res, head))


In [9]:
def tb_callback(name):
        logdir = os.path.join("tensorboard_logs", name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        return tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [10]:
def subset_data(n, X, Y):
    return X[:n, :], Y[:n, :]


# Basic sequential model

We will begin training a basic fully-connected network specified with Keras. This is primarily meant as a test and to define some useful functions (such as those for visualising our outputs). We will start by preprocessing, then defining a model, training and evaluating.

### Data generation
The script `generate_data.py` will open a list of PGN (portable game notation) chess games and run through them, getting from it some state during the game. This state contains information on the location, type and colour of the pieces as well as the legal moves in the position. However, it does not have any information on the strength of those moves, meaning we can only predict the *legal* moves and not rank them.

The games which are generated are saved in a `txt` file as FEN (Forsyth-Edwards Notation) strings, which encapsulate all the information needed to recreate a position in chess. We can use these strings to reconstruct the boards and gain access to the information we need. 

The current file being used was obtained from Lichess and is a set of ~121,000 games played online over the course of 2013. They are a mixture of classical, blitz and bullet games played at a variety of different levels.

In [11]:
with open("generated_data/lichess_db_standard_rated_2013-01.txt", 'r') as f:
    boards = f.readlines()

board_list = [construct_board(i) for i in boards]

In [12]:
# An example of what one of these positions looks like
view_board_colour(board_list[0])

Black to move - turn 7
 R  N        K  B     R 
 P  B  P  P     P     P 
    P        P        P 
                   Q    
          P  P     B    
 P     N                
    P  P        P  P  P 
 R        Q  K     N  R 



## Data processing
Now the data needs to be formed into a reasonable type for training. This section will get all the positions which can be moved to as a list for each game, and all the positions of the pieces for each board as training data. Initally, this won't account for the piece type, which side is to move, or the different types of pieces and moves.

In [13]:
# Uses set notation to get only unique moves
valid_move_list = [list({i for i in board.moves.all_valid}) for board in board_list] 

# Get all the piece types and positions from the location map of each board (training data)
piece_position_list = [{position: piece.kind for position, piece in board.loc_map.items()} for board in board_list]

In [14]:
# Encode the location of any piece as a 1, and all other positions as 0
feature = []
for instance in piece_position_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    feature.append(array.flatten())

In [15]:
# We encode the valid "destination" squares with a 1 and invalid with a 0
target = []
for instance in valid_move_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    target.append(array.flatten())

In [16]:
X_fc_all = np.array(feature)
Y_fc_all = np.array(target)

X_fc, Y_fc = subset_data(1000, X_fc_all, Y_fc_all)

# Split the data using sklearn train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_fc, Y_fc, test_size=0.2)

Beginning with a simple sequential model, which takes in a 1x64 array of features and outputs a prediction at each of the 64 squares.

In [17]:
fc_model_inputs = keras.Input(shape=(64, ))
fc_model = Dense(128, activation='relu')(fc_model_inputs)
fc_model = Dense(256, activation='relu')(fc_model)
fc_model_outputs = Dense(64, activation='sigmoid')(fc_model)

fc_built_model = keras.Model(inputs=fc_model_inputs, outputs=fc_model_outputs)
fc_built_model.compile(
    optimizer='adam', 
    loss='binary_crossentropy',
    metrics=['accuracy', 'binary_accuracy']
)
fc_built_model.summary()

fc_callback = tb_callback("fc_initial_model")

Metal device set to: Apple M1
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
Total params: 57,792
Trainable params: 57,792
Non-trainable params: 0
_________________________________________________________________


2021-12-15 01:35:46.478179: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-15 01:35:46.478611: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-12-15 01:35:46.589591: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-15 01:35:46.589599: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-15 01:35:46.590003: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


In [18]:
fc_built_model.fit(
    X_train, 
    Y_train,
    epochs=100,
    batch_size=5,
    callbacks=[fc_callback]
)

Epoch 1/100


2021-12-15 01:35:46.636777: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-15 01:35:46.639446: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-15 01:35:46.830285: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


  6/160 [>.............................] - ETA: 1s - loss: 0.6893 - accuracy: 0.0333 - binary_accuracy: 0.5417     

2021-12-15 01:35:46.980914: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-15 01:35:46.980923: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-15 01:35:46.988617: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-15 01:35:46.992505: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-12-15 01:35:46.996969: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tensorboard_logs/fc_initial_model_20211215-013546/train/plugins/profile/2021_12_15_01_35_46
2021-12-15 01:35:46.998113: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to tensorboard_logs/fc_initial_model_20211215-013546/train/plugins/profile/2021_12_15_01_35_46/Elliss-MacBook-Air.local.trace.json.gz
2021-12-15 01:35:47.001208: I tensorflow/core/profiler/rpc/client/save_profile.cc:1

160/160 [==============================] - 1s 6ms/step - loss: 0.6041 - accuracy: 0.0050 - binary_accuracy: 0.6829
Epoch 2/100
160/160 [==============================] - 1s 7ms/step - loss: 0.5630 - accuracy: 0.0112 - binary_accuracy: 0.7119
Epoch 3/100
160/160 [==============================] - 1s 6ms/step - loss: 0.5331 - accuracy: 0.0175 - binary_accuracy: 0.7288
Epoch 4/100
160/160 [==============================] - 1s 6ms/step - loss: 0.5074 - accuracy: 0.0137 - binary_accuracy: 0.7437
Epoch 5/100
160/160 [==============================] - 1s 6ms/step - loss: 0.4861 - accuracy: 0.0162 - binary_accuracy: 0.7581
Epoch 6/100
160/160 [==============================] - 1s 6ms/step - loss: 0.4660 - accuracy: 0.0225 - binary_accuracy: 0.7733
Epoch 7/100
160/160 [==============================] - 1s 6ms/step - loss: 0.4509 - accuracy: 0.0200 - binary_accuracy: 0.7834
Epoch 8/100
160/160 [==============================] - 1s 6ms/step - loss: 0.4320 - accuracy: 0.0213 - binary_accuracy: 0.7

160/160 [==============================] - 1s 6ms/step - loss: 0.0185 - accuracy: 0.0850 - binary_accuracy: 0.9997
Epoch 66/100
160/160 [==============================] - 1s 7ms/step - loss: 0.0169 - accuracy: 0.0950 - binary_accuracy: 0.9999
Epoch 67/100
160/160 [==============================] - 1s 6ms/step - loss: 0.0157 - accuracy: 0.0887 - binary_accuracy: 0.9997
Epoch 68/100
160/160 [==============================] - 1s 6ms/step - loss: 0.0146 - accuracy: 0.0912 - binary_accuracy: 0.9998
Epoch 69/100
160/160 [==============================] - 1s 6ms/step - loss: 0.0133 - accuracy: 0.0912 - binary_accuracy: 0.9999
Epoch 70/100
160/160 [==============================] - 1s 7ms/step - loss: 0.0124 - accuracy: 0.0900 - binary_accuracy: 1.0000
Epoch 71/100
160/160 [==============================] - 1s 6ms/step - loss: 0.0117 - accuracy: 0.0938 - binary_accuracy: 0.9999
Epoch 72/100
160/160 [==============================] - 1s 6ms/step - loss: 0.0106 - accuracy: 0.0912 - binary_accura

In [19]:
fc_built_model.evaluate(
    X_test, 
    Y_test,
    callbacks=[fc_callback],
)

1/7 [===>..........................] - ETA: 1s - loss: 2.1875 - accuracy: 0.1250 - binary_accuracy: 0.7085

2021-12-15 01:37:24.740183: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 0s 21ms/step - loss: 2.5528 - accuracy: 0.0500 - binary_accuracy: 0.6814


[2.5528109073638916, 0.04999999701976776, 0.6814062595367432]

In [20]:
%tensorboard --logdir tensorboard_logs

## Visualising the output
Now both the raw output of the model and the binarised version can be displayed interactively.

In [21]:
# From https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
interact(
    generate_plots,
    model = fixed(fc_built_model),
    x = fixed(X_test),
    y = fixed(Y_test),
    n=widgets.IntSlider(min=0, max=199, step=5, value=0),
    target=False,
    prediction=False,
    raw_prediction=False,
    diff=True
)

interactive(children=(IntSlider(value=0, description='n', max=199, step=5), Checkbox(value=False, description=…

<function __main__.generate_plots(n, model=None, x=None, y=None, target=True, prediction=True, raw_prediction=True, diff=True)>

This widget lets us look through the validation set predictions and compare what the model is outputting compared to what the equivilant label is. What we can begin to see from here is that the model does have some predictive power on pieces which do not move great distances up and down the board. However for pieces that can travel the entire diagonal, vertical or horizontal distance of the board (such as rooks, bishops and queens) there is a high margin of error and these motions are rarely predicted at all, indicating they are tough to learn.

This may also be indicitve of some bias in the training data - almost every position has pawns which can be moved in it whereas the major pieces are more often closed in and obscured behind other pieces. This imbalance may contribute to not learning how the pieces move properly.

## Hyperparameters

This model has many hyperparemters already, including:
- The number of epochs
- The size of each batch
- The optimizer used
- The number of parameters in each hidden layer
- The number of hidden layers

It is also more than reasonable to think of adding other optimization techniques such as regularization.
To help optimize these parameters we can use a grid search algorithm. 

### Tuning the model architecture

The network design is a very important factor and so its useful to get an idea of what model design will perform well. This grid search will fix certain parameters for speed and then search a space of different model designs in a coarse grid to get a picture for which architecture to use when tuning other parameters.

In [22]:
def create_fc_model(
        n_hidden_layers, 
        hidden_dim,
        optimizer,
        batch_size,
        n_epochs,
        output_activation,
        dropout_rate
        ):
    # Create a model  
    fc_model_inputs = keras.Input(shape=(64, ))
    fc_model = Dense(hidden_dim, activation='relu')(fc_model_inputs)
    for i in range(n_hidden_layers-1):
        fc_model = Dense(hidden_dim, activation='relu')(fc_model)
        fc_model = Dropout(dropout_rate)(fc_model)
        
    fc_model_outputs = Dense(64, activation=output_activation)(fc_model)

    model = keras.Model(inputs=fc_model_inputs, outputs=fc_model_outputs)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', 'binary_accuracy'])
    return model

In [23]:
fc_layers_param_grid = {
        "n_hidden_layers": [1, 2, 4, 8],
        "hidden_dim": [32, 64, 128, 256, 512],
        "optimizer": ["adam"],
        "batch_size": [5],
        "n_epochs": [250],
        "output_activation": ['sigmoid'],
        "dropout_rate": [0]
}

X_fc_grid, Y_fc_grid = subset_data(2000, X_fc_all, Y_fc_all)


fc_layers_model = KerasClassifier(build_fn=create_fc_model)
fc_layers_grid = GridSearchCV(estimator=fc_layers_model, param_grid=fc_layers_param_grid, n_jobs=-1, cv=3, verbose=1)
fc_layers_result = fc_layers_grid.fit(X_fc_grid, Y_fc_grid)



Fitting 3 folds for each of 20 candidates, totalling 60 fits
Init Plugin
Init Plugin
Init Plugin
Init Graph Optimizer
Init Kernel
Init Plugin
Init Graph Optimizer
Init Kernel
Init Graph Optimizer
Init Kernel
Init Graph Optimizer
Init Kernel
Init Plugin
Init Graph Optimizer
Init Kernel
Init Plugin
Init Graph Optimizer
Init Kernel
Init Plugin
Init Graph Optimizer
Init Kernel
Init Plugin
Init Graph Optimizer
Init Kernel
Metal device set to: Apple M1


2021-12-15 01:37:34.581281: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-15 01:37:34.581502: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-12-15 01:37:34.600494: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-15 01:37:34.600668: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL,

Metal device set to: Apple M1
Metal device set to: Apple M1
Metal device set to: Apple M1
Metal device set to: Apple M1
Metal device set to: Apple M1
Metal device set to: Apple M1
Metal device set to: Apple M1


2021-12-15 01:37:35.102080: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-15 01:37:35.102751: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-15 01:37:35.105462: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-15 01:37:35.105495: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-15 01:37:35.105751: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-15 01:37:35.108912: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-15 01:37:35.123532: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-15 01

267/267 [==============================] - 9s 30ms/step - loss: 0.6153 - accuracy: 0.0060 - binary_accuracy: 0.6771


...

 20/134 [===>..........................] - ETA: 1s - loss: 0.6001 - accuracy: 0.0000e+00 - binary_accuracy: 0.6823

2021-12-15 01:39:23.625671: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


  9/400 [..............................] - ETA: 2s - loss: 0.6759 - accuracy: 0.0000e+00 - binary_accuracy: 0.5847  

2021-12-15 01:39:25.793215: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


400/400 [==============================] - 3s 6ms/step - loss: 0.5755 - accuracy: 0.0060 - binary_accuracy: 0.7074


In [24]:
disp_grid(fc_layers_result)

Best: 0.011998505486796299 using {'batch_size': 5, 'dropout_rate': 0, 'hidden_dim': 256, 'n_epochs': 250, 'n_hidden_layers': 2, 'optimizer': 'adam', 'output_activation': 'sigmoid'}
      Mean    Standard deviation    batch_size    dropout_rate    hidden_dim    n_epochs    n_hidden_layers  optimizer    output_activation
----------  --------------------  ------------  --------------  ------------  ----------  -----------------  -----------  -------------------
0.006                0.00244827              5               0            32         250                  1  adam         sigmoid
0.0054995            0.00141192              5               0            32         250                  2  adam         sigmoid
0.00300075           0.00122689              5               0            32         250                  4  adam         sigmoid
0.003499             0.00186909              5               0            32         250                  8  adam         sigmoid
0.00550175       

### Tuning opsimizer, batch size and training duration

These other parameters can now be explored since there is a more solid foundation of the model architecture. Using the previous result its now possible to look at the next iteration of parameters. It is indicated that a model with 1 hidden layer and 256 nodes performs best, so this will be the choice for the next stage.

In [25]:
op_param_grid = {
        "n_hidden_layers": [1],
        "hidden_dim": [256],
        "optimizer": ["SGD", "adam", "adamax"],
        "batch_size": [5, 20, 50],
        "n_epochs": [100, 250, 500, 750],
        "output_activation": ['sigmoid'],
        "dropout_rate": [0]
}
fc_op_model = KerasClassifier(build_fn=create_fc_model, verbose=1)
fc_op_grid = GridSearchCV(estimator=fc_op_model, param_grid=op_param_grid, n_jobs=-1, cv=3, verbose=2)
fc_op_result = fc_op_grid.fit(X_fc_grid, Y_fc_grid)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


2021-12-15 01:39:28.707290: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:39:28.721354: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:39:28.722151: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:39:28.738546: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:39:28.739570: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:39:28.782574: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


127/267 [=============>................] - ETA: 3s - loss: 0.6101 - accuracy: 0.0079 - binary_accuracy: 0.6856

2021-12-15 01:39:32.189155: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


267/267 [==============================] - 7s 25ms/step - loss: 0.5912 - accuracy: 0.0090 - binary_accuracy: 0.7020


...

14/14 [==============================] - 0s 20ms/step - loss: 0.6028 - accuracy: 0.0075 - binary_accuracy: 0.6940


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
36 fits failed out of a total of 108.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/wrappers/scikit_learn.py", line 220, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/opt/h

 20/100 [=====>........................] - ETA: 0s - loss: 0.7121 - accuracy: 0.0300 - binary_accuracy: 0.4764   

2021-12-15 01:40:28.924998: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


100/100 [==============================] - 1s 5ms/step - loss: 0.7082 - accuracy: 0.0165 - binary_accuracy: 0.4866


In [26]:
disp_grid(fc_op_result)

Best: 0.01650000906859835 using {'batch_size': 20, 'dropout_rate': 0, 'hidden_dim': 256, 'n_epochs': 500, 'n_hidden_layers': 1, 'optimizer': 'SGD', 'output_activation': 'sigmoid'}
        Mean    Standard deviation    batch_size    dropout_rate    hidden_dim    n_epochs    n_hidden_layers  optimizer    output_activation
------------  --------------------  ------------  --------------  ------------  ----------  -----------------  -----------  -------------------
  0.009499             0.00282436              5               0           256         100                  1  SGD          sigmoid
  0.00749775           0.00367056              5               0           256         100                  1  adam         sigmoid
nan                  nan                       5               0           256         100                  1  adamax       sigmoid
  0.015499             0.00627962              5               0           256         250                  1  SGD          sigmoid
  0.00

The results here indicate that the stochastic gradient descent optimizer with a small batch size and 500 epochs will give us the best performance on a testing split. We can now look at using this result to tune the learning rate and momentum parameters, as well as any regularisation

### Tuning optimizer parameters and output activation function

Now the optimizer hyperamarameters, activation function and regularization can be tuned

In [27]:
act_param_grid = {
        "n_hidden_layers": [1],
        "hidden_dim": [256],
        "optimizer": ["SGD"],
        "batch_size": [5],
        "n_epochs": [750],
        "output_activation": ['sigmoid', 'softmax'],
        "dropout_rate": [0, 0.4, 0.6, 0.8]
}
fc_act_model = KerasClassifier(build_fn=create_fc_model, verbose=1)
fc_act_grid = GridSearchCV(estimator=fc_act_model, param_grid=act_param_grid, n_jobs=-1, cv=3, verbose=2)
fc_act_result = fc_act_grid.fit(X_fc_grid, Y_fc_grid)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


2021-12-15 01:40:30.009760: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:40:30.039044: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:40:30.048017: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:40:30.075759: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:40:30.076865: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:40:30.099265: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:40:30.099640: I tensorflow/core/grappler/optimizers/cust

[CV] END batch_size=50, dropout_rate=0, hidden_dim=256, n_epochs=750, n_hidden_layers=1, optimizer=adam, output_activation=sigmoid; total time=   2.2s
  8/267 [..............................] - ETA: 6s - loss: 0.6896 - accuracy: 0.0250 - binary_accuracy: 0.5312[CV] END batch_size=50, dropout_rate=0, hidden_dim=256, n_epochs=500, n_hidden_layers=1, optimizer=adam, output_activation=sigmoid; total time=   2.4s
[CV] END batch_size=50, dropout_rate=0, hidden_dim=256, n_epochs=750, n_hidden_layers=1, optimizer=adamax, output_activation=sigmoid; total time=   0.6s
[CV] END batch_size=50, dropout_rate=0, hidden_dim=256, n_epochs=750, n_hidden_layers=1, optimizer=adamax, output_activation=sigmoid; total time=   0.7s
  1/267 [..............................] - ETA: 3:20 - loss: 0.7036 - accuracy: 0.0000e+00 - binary_accuracy: 0.6938[CV] END batch_size=50, dropout_rate=0, hidden_dim=256, n_epochs=500, n_hidden_layers=1, optimizer=adam, output_activation=sigmoid; total time=   2.4s
[CV] END batch_

2021-12-15 01:40:38.200768: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:40:38.269949: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:40:38.321455: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 01:40:38.373186: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


  1/134 [..............................] - ETA: 32s - loss: 0.6685 - accuracy: 0.0000e+00 - binary_accuracy: 0.5844

...

134/134 [==============================] - 3s 19ms/step - loss: 0.6814 - accuracy: 0.0075 - binary_accuracy: 0.6887


2021-12-15 01:41:06.228329: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


 18/400 [>.............................] - ETA: 2s - loss: 0.6969 - accuracy: 0.0111 - binary_accuracy: 0.6837    

2021-12-15 01:41:07.604665: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


400/400 [==============================] - 2s 5ms/step - loss: 0.6828 - accuracy: 0.0050 - binary_accuracy: 0.6955


In [28]:
disp_grid(fc_act_result)

Best: 0.024505015462636948 using {'batch_size': 5, 'dropout_rate': 0, 'hidden_dim': 256, 'n_epochs': 750, 'n_hidden_layers': 1, 'optimizer': 'SGD', 'output_activation': 'softmax'}
      Mean    Standard deviation    batch_size    dropout_rate    hidden_dim    n_epochs    n_hidden_layers  optimizer    output_activation
----------  --------------------  ------------  --------------  ------------  ----------  -----------------  -----------  -------------------
0.00850025            0.00187111             5             0             256         750                  1  SGD          sigmoid
0.024505              0.00709194             5             0             256         750                  1  SGD          softmax
0.0124975             0.00352846             5             0.4           256         750                  1  SGD          sigmoid
0.00599925            0.00211867             5             0.4           256         750                  1  SGD          softmax
0.0100003         

### Result

From this hyperparameter optimisation we can see that the best model learned will have:
- Batch size = 5
- Dropout rate = 0
- Hidden layers = 1
- Nodes in hidden layers = 256
- Optimizer function = SGD
- Output activation function = sigmoid
We can now use this data to cross validate a model on the entire dataset.

In [29]:
fc_model_params = fc_act_result.best_params_
fc_model = create_fc_model(**fc_model_params)
fc_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               16640     
_________________________________________________________________
dense_11 (Dense)             (None, 64)                16448     
Total params: 33,088
Trainable params: 33,088
Non-trainable params: 0
_________________________________________________________________


In [ ]:
N_folds = 3
N_instances = 4000

kfold = KFold(n_splits = N_folds)
metrics = []
fold = 1
X_fc_final, Y_fc_final = subset_data(N_instances, X_fc_all, Y_fc_all)
for train_idx, test_idx in kfold.split(X_fc_final, Y_fc_final):
    cb = tb_callback(f"fc_model_fold_{fold}") 
    mdl = create_fc_model(**fc_model_params)
    mdl.fit(
        X_fc_all[train_idx],
        Y_fc_all[train_idx],
        epochs=fc_model_params["n_epochs"],
        batch_size=fc_model_params["batch_size"],
        validation_split=0.1,
        callbacks=[cb]
    )
    
    ev = mdl.evaluate(
        X_fc_all[test_idx],
        Y_fc_all[test_idx],
        callbacks=[cb]
    )
    
    metrics.append(ev)
    fold += 1

Epoch 1/750


2021-12-15 01:41:09.838815: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-15 01:41:09.838833: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-15 01:41:09.839579: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-12-15 01:41:10.007187: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


 15/480 [..............................] - ETA: 6s - loss: 0.7052 - accuracy: 0.0667 - binary_accuracy: 0.6727   

2021-12-15 01:41:10.221849: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-15 01:41:10.221859: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-15 01:41:10.272606: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-15 01:41:10.277581: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-12-15 01:41:10.283904: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tensorboard_logs/fc_model_fold_1_20211215-014109/train/plugins/profile/2021_12_15_01_41_10
2021-12-15 01:41:10.285255: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to tensorboard_logs/fc_model_fold_1_20211215-014109/train/plugins/profile/2021_12_15_01_41_10/Elliss-MacBook-Air.local.trace.json.gz
2021-12-15 01:41:10.289227: I tensorflow/core/profiler/rpc/client/save_profile.cc:137

480/480 [==============================] - ETA: 0s - loss: 0.6877 - accuracy: 0.0279 - binary_accuracy: 0.6946

...

Epoch 234/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4880 - accuracy: 0.0146 - binary_accuracy: 0.6946 - val_loss: 0.5092 - val_accuracy: 0.0075 - val_binary_accuracy: 0.6973
Epoch 235/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4878 - accuracy: 0.0158 - binary_accuracy: 0.6946 - val_loss: 0.5091 - val_accuracy: 0.0075 - val_binary_accuracy: 0.6973
Epoch 236/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4876 - accuracy: 0.0154 - binary_accuracy: 0.6946 - val_loss: 0.5091 - val_accuracy: 0.0075 - val_binary_accuracy: 0.6973
Epoch 237/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4874 - accuracy: 0.0150 - binary_accuracy: 0.6946 - val_loss: 0.5089 - val_accuracy: 0.0075 - val_binary_accuracy: 0.6973
Epoch 238/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4873 - accuracy: 0.0150 - binary_accuracy: 0.6946 - val_loss: 0.5089 - val_accuracy: 0.0075 - val_binary_accuracy: 0.

Epoch 275/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4817 - accuracy: 0.0167 - binary_accuracy: 0.6946 - val_loss: 0.5063 - val_accuracy: 0.0075 - val_binary_accuracy: 0.6973
Epoch 276/750
480/480 [==============================] - 5s 9ms/step - loss: 0.4816 - accuracy: 0.0183 - binary_accuracy: 0.6946 - val_loss: 0.5062 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 277/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4814 - accuracy: 0.0167 - binary_accuracy: 0.6946 - val_loss: 0.5062 - val_accuracy: 0.0075 - val_binary_accuracy: 0.6973
Epoch 278/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4813 - accuracy: 0.0171 - binary_accuracy: 0.6946 - val_loss: 0.5061 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 279/750
480/480 [==============================] - 5s 10ms/step - loss: 0.4812 - accuracy: 0.0167 - binary_accuracy: 0.6946 - val_loss: 0.5061 - val_accuracy: 0.0112 - val_binary_accuracy: 0

Epoch 316/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4769 - accuracy: 0.0163 - binary_accuracy: 0.6946 - val_loss: 0.5045 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 317/750
480/480 [==============================] - 5s 10ms/step - loss: 0.4768 - accuracy: 0.0175 - binary_accuracy: 0.6946 - val_loss: 0.5045 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 318/750
480/480 [==============================] - 5s 10ms/step - loss: 0.4767 - accuracy: 0.0163 - binary_accuracy: 0.6946 - val_loss: 0.5044 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 319/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4766 - accuracy: 0.0163 - binary_accuracy: 0.6946 - val_loss: 0.5047 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 320/750
480/480 [==============================] - 5s 10ms/step - loss: 0.4765 - accuracy: 0.0167 - binary_accuracy: 0.6946 - val_loss: 0.5045 - val_accuracy: 0.0112 - val_binary_accuracy:

Epoch 357/750
480/480 [==============================] - 4s 8ms/step - loss: 0.4731 - accuracy: 0.0171 - binary_accuracy: 0.6946 - val_loss: 0.5032 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 358/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4730 - accuracy: 0.0167 - binary_accuracy: 0.6946 - val_loss: 0.5032 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 359/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4729 - accuracy: 0.0171 - binary_accuracy: 0.6946 - val_loss: 0.5031 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 360/750
480/480 [==============================] - 5s 10ms/step - loss: 0.4728 - accuracy: 0.0167 - binary_accuracy: 0.6946 - val_loss: 0.5032 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 361/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4727 - accuracy: 0.0167 - binary_accuracy: 0.6946 - val_loss: 0.5034 - val_accuracy: 0.0112 - val_binary_accuracy: 0

Epoch 398/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4699 - accuracy: 0.0179 - binary_accuracy: 0.6946 - val_loss: 0.5026 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 399/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4698 - accuracy: 0.0179 - binary_accuracy: 0.6946 - val_loss: 0.5024 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 400/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4697 - accuracy: 0.0179 - binary_accuracy: 0.6946 - val_loss: 0.5025 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 401/750
480/480 [==============================] - 4s 8ms/step - loss: 0.4696 - accuracy: 0.0179 - binary_accuracy: 0.6946 - val_loss: 0.5025 - val_accuracy: 0.0112 - val_binary_accuracy: 0.6973
Epoch 402/750
480/480 [==============================] - 4s 9ms/step - loss: 0.4695 - accuracy: 0.0183 - binary_accuracy: 0.6946 - val_loss: 0.5025 - val_accuracy: 0.0112 - val_binary_accuracy: 0.

# Convolutional design

We can also approach this problem with a convolutional network. These are designed for image processing and operate by sliding a kernel over an image, performing a "convolution" of the kernel on the data. It is particuarly good at identifying patterns, so let us test this out!

## Data manipulation

For this model, the data will be separated into two layers; one for all the white pieces and one for the black pieces. The target will remain unchanged and there will still be no distinction between the pieces.

To get the data in the correct format, we can construct a $8\times8\times2$ numpy array to feed our model. The pieces which are moving will be provided in the 0th index of axis 2 (the array is structured [axis0, axis1, axis2]) and those not moving will be at index 1.

In [ ]:
conv_move_list = [list({i for i in board.moves.all_valid}) for board in board_list]
conv_position_list = [
    (dict(board.loc_map), board.to_move) 
    for board in board_list
]

In [ ]:
# Encode the location of any piece as a 1, and all other positions as 0
conv_feature = []
for instance in conv_position_list:
    # Set up empty array
    array = np.zeros((8, 8, 2))
    
    # Assign white and black to their correct layers
    if instance[1] == WHITE:
        # White to move
        WHITE_LAYER = 1
        BLACK_LAYER = 0
    else:
        # Black to move
        BLACK_LAYER = 1
        WHITE_LAYER = 0
    
    # Mark the correct locations in the matrix
    for location, piece in instance[0].items():
        if piece.colour == WHITE:
            array[location.i, location.j, WHITE_LAYER] = 1
        else:
            array[location.i, location.j, BLACK_LAYER] = 1

    conv_feature.append(array)

In [ ]:
# We encode the valid "destination" squares with a 1 and invalid with a 0
conv_target = []
for instance in conv_move_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    conv_target.append(array.flatten())

In [ ]:
X_cnv1_all = np.array(conv_feature)
Y_cnv1_all = np.array(conv_target)

X_cnv1_subset, Y_cnv1_subset = subset_data(1000, X_cnv1_all, Y_cnv1_all)

# Split the data using sklearn train_test_split
X_cnv1_train, X_cnv1_test, Y_cnv1_train, Y_cnv1_test = train_test_split(
    X_cnv1_subset, 
    Y_cnv1_subset, 
    test_size=0.2
)

### Training a model

Now the data is in a format that contains much more information in a way that convolutional netoworks can process, we can define a convolutional model architecture (again using the Keras functional API) to verify if this model can perform any better.

The head neural network at the end will share a design with the result of the fully-connected grid search, however that result is not transferable and so a proper grud search for this model will be carried out further down.

In [ ]:
conv_1_input = keras.Input(shape=(8, 8, 2))
cnv_1 = Conv2D(8, (3, 3), activation='relu')(conv_1_input)
cnv_1 = Conv2D(8, (3, 3), activation='relu')(cnv_1)
cnv_1 = Conv2D(8, (3, 3), activation='relu')(cnv_1)
cnv_1 = Flatten()(cnv_1)
cnv_1 = Dense(256, activation='relu')(cnv_1)
conv_1_outputs = Dense(64, activation='sigmoid')(cnv_1)

cnv1_model = keras.Model(inputs=conv_1_input, outputs=conv_1_outputs)
cnv1_model.compile(
    optimizer="SGD",
    loss="binary_crossentropy",
    metrics=["accuracy", "binary_accuracy"]
)
cnv1_model.summary()

cnv1_tb_callback = tb_callback("cnv1_base")

In [ ]:
cnv1_model.fit(
    X_cnv1_train, 
    Y_cnv1_train,
    epochs=750,
    batch_size=5,
    validation_split=0.1,
    callbacks=[cnv1_tb_callback]
)

In [ ]:
cnv1_model.evaluate(X_cnv1_test, Y_cnv1_test, callbacks=[cnv1_tb_callback])

The binary accuracy of this model does outperform the previous dense network used before, which is a promising result for this technique.

In [ ]:
interact(
    generate_plots,
    model = fixed(cnv1_model),
    x = fixed(X_cnv1_test),
    y = fixed(Y_cnv1_test),
    n=widgets.IntSlider(min=0, max=199, step=5, value=0),
    target=False,
    prediction=False,
    raw_prediction=False,
    diff=True
)

From these modified visualisation functions it is clear that there is greater performance by this model. The added implicit information on which pieces are moving, combined with the use of convolutional filters to properly extract features from the training data. Seeing a jump in binary accuracy as well as in many cases much more confident predictions from the model is indicitive that this approach is superior, due to the convolutional feature extraction layers

The behaviour of long range pieces is however still an issue for these systems.

# Convolutional network with higher input dimensionallity

It is now not unreasonable to think that the networks performance would be increased significantly if it were to be aware of which pieces are at which location; this is after all the most important deciding factor as to how they may move.

An approach to integrating this data is the following architecture;
- Input data (8 x 8 x 6), where there is one index on axis 2 for each of the five piece types (King, Queen, Rook, Knight, Bishop, Pawn) for the moving pieces, and another for the following
- The final layer of the input tensor will contain the position of the pieces which are not to move. Since the information on which type of piece these are is generally less important to predicting the moves, for now this can be left out.
- Additionally, passing the fully connected layer some information about whether the position is check for example may increase accuracy some more, since it will be able to tell when certain moves which usually are valid would not be valid. This information needs to reach the fully connected network but is not part of the convolutional network, so for this section the Keras functional API will be more appropriate as it offers more control.





## Data processing

In [ ]:
# Encode the location of any piece as a 1, and all other positions as 0
conv_feature = []
piece_index_map = {
    'K': 0,
    'Q': 1,
    'R': 2,
    'B': 3,
    'N': 4,
    'P': 5
}
for instance in conv_position_list:
    # Set up empty array
    array = np.zeros((8, 8, 7))
    
    # Assign white and black to their correct layers
    if instance[1] == WHITE:
        # White to move
        MOVING_COLOUR = 1
    else:
        # Black to move
        MOVING_COLOUR = 0
    
    # Mark the correct locations in the matrix
    for location, piece in instance[0].items():
        if piece.colour == MOVING_COLOUR:
            array[location.i, location.j, piece_index_map[piece.kind]] = 1
        else:
            array[location.i, location.j, 6] = 1

    conv_feature.append(array)

In [ ]:
# We encode the valid "destination" squares with a 1 and invalid with a 0
conv_target = []
for instance in conv_move_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    conv_target.append(array.flatten())

In [ ]:
X_cnv2_all = np.array(conv_feature)
Y_cnv2_all = np.array(conv_target)

# Split the data using sklearn train_test_split
X_cnv2_train, X_cnv2_test, Y_cnv2_train, Y_cnv2_test = train_test_split(X_cnv2_all, Y_cnv2_all, test_size=0.2)

## Create a model

This model architecture will be roughly similar to that of the previous, however this time we will perform an exploratory grid search on a subset of the data to establish the best hyperparameters.

In [ ]:
def create_cnv2_model(
        n_conv_layers,
        n_kernels,
    
        n_hidden_layers, 
        hidden_dim,
    
        optimizer,
        batch_size,
        n_epochs,
        dropout_rate,
        learning_rate,
        momentum,
        ):
    conv_2_input = keras.Input(shape=(8, 8, 7))
    cnv_2 = Conv2D(n_kernels, (3, 3), activation='relu')(conv_2_input)
    for i in range(n_conv_layers - 1):
        cnv_2 = Conv2D(n_kernels, (3, 3), activation='relu')(cnv_2)

    cnv_2 = Flatten()(cnv_2)
    for i in range(n_hidden_layers-1):
        cnv_2 = Dense(hidden_dim, activation='relu')(cnv_2)
        cnv_2 = Dropout(dropout_rate)(cnv_2)
    conv_2_outputs = Dense(64, activation='sigmoid')(cnv_2)
    
    opt = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)

    conv_model_2 = keras.Model(inputs=conv_2_input, outputs=conv_2_outputs)
    conv_model_2.compile(
        optimizer=opt,
        loss="binary_crossentropy",
        metrics=["accuracy", "binary_accuracy"]
    )
    return conv_model_2

### Perform a hyperparameter grid search

In [ ]:
cnv2_param_grid = {
        "n_conv_layers": [1],
        "n_kernels": [10, 15],
        "n_hidden_layers": [3, 5],
        "hidden_dim": [16, 32, 64],
        "optimizer": ["SGD"],
        "batch_size": [5],
        "n_epochs": [750, 1000],
        "dropout_rate": [0],
        "learning_rate": [0.01, 0.05, 0.1],
        "momentum": [0, 0.2]
}


X_cnv2_grid, Y_cnv2_grid = subset_data(1000, X_cnv2_all, Y_cnv2_all)


cnv2_model = KerasClassifier(build_fn=create_cnv2_model)
cnv2_grid = GridSearchCV(estimator=cnv2_model, param_grid=cnv2_param_grid, n_jobs=-1, cv=3, verbose=2)
cnv2_result = cnv2_grid.fit(X_cnv2_grid, Y_cnv2_grid)



In [ ]:
disp_grid(cnv2_result)

In [ ]:
cnv2_model = create_cnv2_model(**cnv2_result.best_params_)
cnv2_model.summary()
cnv2_tb_callback = tb_callback("cnv2")

In [ ]:
N_instances = 2000

X_cnv2_subset, Y_cnv2_subset = subset_data(N_instances, X_cnv2_all, Y_cnv2_all)

X_cnv2_train, X_cnv2_test, Y_cnv2_train, Y_cnv2_test = train_test_split(X_cnv2_subset, Y_cnv2_subset, test_size=0.2)

cnv2_model.fit(
    X_cnv2_train, 
    Y_cnv2_train,
    validation_split=0.1,
    epochs=cnv2_result.best_params_["n_epochs"],
    batch_size=cnv2_result.best_params_["batch_size"],
    callbacks=[cnv2_tb_callback]
)

In [ ]:
conv_model_2.evaluate(X_cnv2_test, Y_cnv2_test)

## Visualise the output

In [ ]:
# From https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
interact(
    generate_plots,
    model = fixed(cnv2_model),
    x = fixed(X_cnv2_test),
    y = fixed(Y_cnv2_test),
    n=widgets.IntSlider(min=0, max=199, step=5, value=0),
    target=False,
    prediction=False,
    raw_prediction=False,
    diff=True
)